<a href='http://moncoachdata.com'><img src= 'https://moncoachdata.com/wp-content/uploads/MonCoachData-cours/MonCoachData.jpg' width=400/></a>


---


<center><em>Copyright MonCoachData (tous droits réservés)</em></center>
<center><em>Pour plus d'informations, visitez notre site <a href='http://moncoachdata.com'>moncoachdata.com</a></em></center>



---

# Introduction à la Validation Croisée

Dans cette série de vidéos, nous allons étudierr plus profondément les différentes méthodes de validation croisée (cros validation). Nous discuterons également de la philosophie générale de la validation croisée. Un super guide issu de la documentation officielle peut être trouvé ici : https://scikit-learn.org/stable/modules/cross_validation.html

## Importations

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Exemple de Données

In [ ]:
df = pd.read_csv("Advertising.csv")

In [ ]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


## Procédure du Train | Test Split 

0. Nettoyer et ajuster les données si nécessaire pour X et y
1. Fractionner les données en Train/Test pour X et y.
2. Fit/Adapter le scaler sur les données X d'entraînement (mise à l'échelle)
3. Mettre à l'échelle les données X de test
4. Créer un modèle
5. Fit/entraîner le modèle sur les données d'entraînement X
6. Évaluer le modèle sur les données de test X (en créant des prédictions et en les comparant à y_test)
7. Ajuster les paramètres si nécessaire et répétez les étapes 5 et 6...

In [ ]:
## CRÉER X et y
X = df.drop('sales',axis=1)
y = df['sales']

# TRAIN TEST SPLIT / Fractionnement Entraînement Test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# SCALE DATA / Mise à l'échelle des données
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

**Création du modèle**

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
# Poor Alpha Choice on purpose!
model = Ridge(alpha=100)

In [ ]:
model.fit(X_train,y_train)

Ridge(alpha=100)

In [ ]:
y_pred = model.predict(X_test)

**Évaluation du modèle**

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(y_test,y_pred)

7.34177578903413

**Adjustement des paramètres et Ré-évaluation du modèle mis à jour**

In [ ]:
model = Ridge(alpha=1)

In [ ]:
model.fit(X_train,y_train)

Ridge(alpha=1)

In [ ]:
y_pred = model.predict(X_test)

**De nouveau évaluation**

In [ ]:
mean_squared_error(y_test,y_pred)

2.319021579428752

Beaucoup mieux ! Nous pourrions répéter ceci jusqu'à ce que nous soyons satisfaits des mesures/metrics de performance. (Nous avons montré précédemment que RidgeCV peut faire cela pour nous, mais le but de cette leçon est de généraliser le processus CV --Cross Validation-- pour tout modèle).

## Procédure Train | Validation | Test Split

Cet ensemble est souvent appelé ensemble "d'attente" (ou Hold-out), car vous ne devez pas ajuster les paramètres en fonction de l'ensemble de Test final, mais l'utiliser *uniquement* pour rendre compte des performances finales attendues du modèle.

0. Nettoyer et ajuster les données si nécessaire pour X et y.
1. Séparer les données en Train/Validation/Test pour X et y.
2. Fit/adapter le scaler sur les données X d'entraînement (mise à l'échelles)
3. Mettre à l'échelle les données d'évaluation X
4. Créer un modèle
5. Fit/entraîner le modèle sur les données d'entraînement X
6. Évaluer le modèle sur les données d'évaluation X (en créant des prédictions et en les comparant à y_eval)
7. Ajuster les paramètres si nécessaire et répétez les étapes 5 et 6.
8. Obtenir les metrics finales sur l'ensemble de test (il n'est pas/plus permis de revenir en arrière et d'ajuster les hyper-paramètress du modèle après cela !)

In [ ]:
## CRÉER X et y
X = df.drop('sales',axis=1)
y = df['sales']

In [ ]:
######################################################################################
#### DOUBLE SPLIT ! Ici nous créons les ensembles TRAIN | VALIDATION | TEST  #########
######################################################################################
from sklearn.model_selection import train_test_split

# 70% des données sont des donnéess d'entraînement/Training, le reste étant de 30%
X_train, X_OTHER, y_train, y_OTHER = train_test_split(X, y, test_size=0.3, random_state=101)

# Les 30% restants sont divisés en sets d'évaluation et test
# Chacun représentant 15% de la taille du datset d'origine
X_eval, X_test, y_eval, y_test = train_test_split(X_OTHER, y_OTHER, test_size=0.5, random_state=101)

In [ ]:
# SCALE DATA / Mise à l'échelle des données
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_eval = scaler.transform(X_eval)
X_test = scaler.transform(X_test)

**Création du modèle**

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
# Choix d'un mauvais alpha intentionnel !
model = Ridge(alpha=100)

In [ ]:
model.fit(X_train,y_train)

Ridge(alpha=100)

In [ ]:
y_eval_pred = model.predict(X_eval)

**Évaluation**

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(y_eval,y_eval_pred)

7.320101458823871

**Adjuster les paramètres ré-évaluer**

In [ ]:
model = Ridge(alpha=1)

In [ ]:
model.fit(X_train,y_train)

Ridge(alpha=1)

In [ ]:
y_eval_pred = model.predict(X_eval)

**Nouvelle évaluation**

In [ ]:
mean_squared_error(y_eval,y_eval_pred)

2.383783075056986

**Évaluation finale (on ne peut plus modifier les paramètres après cela !)**

In [ ]:
y_final_test_pred = model.predict(X_test)

In [ ]:
mean_squared_error(y_test,y_final_test_pred)

2.2542600838005176

---
# Validation Croisée avec cross_val_score

<img src="https://moncoachdata.com/wp-content/uploads/MonCoachData-cours/grid_search_cross_validation.png">

In [ ]:
## CRÉER X et y
X = df.drop('sales',axis=1)
y = df['sales']

# TRAIN TEST SPLIT / Fractionnement Entraînement Test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# SCALE DATA / Mise à l'échelle des données
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
model = Ridge(alpha=100)

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
# Options de SCORING:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_val_score(model,X_train,y_train,
                         scoring='neg_mean_squared_error',cv=5)

In [ ]:
scores

array([ -9.32552967,  -4.9449624 , -11.39665242,  -7.0242106 ,
        -8.38562723])

In [ ]:
# Moyenne des scores MSE (on remet en positif)
abs(scores.mean())

8.215396464543607

**Ajuster le modèe en fonction des metrics**

In [ ]:
model = Ridge(alpha=1)

In [ ]:
# Options de SCORING:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_val_score(model,X_train,y_train,
                         scoring='neg_mean_squared_error',cv=5)

In [ ]:
# Moyenne des scores MSE (on remet en positif))
abs(scores.mean())

3.344839296530695

**Évaluation finale (on ne peut plus modifier les paramètres après cela !)**

In [ ]:
# Il faut d'abord ajuster le modèle !
model.fit(X_train,y_train)

Ridge(alpha=1)

In [ ]:
y_final_test_pred = model.predict(X_test)

In [ ]:
mean_squared_error(y_test,y_final_test_pred)

2.319021579428752

---
# Validation Croisée avec cross_validate

La fonction cross_validate diffère de cross_val_score de deux façons :

- Elle permet de spécifier plusieurs metrics pour l'évaluation.

- Elle renvoie un dictionnaire (dict) contenant les fit-times, les score-times (et éventuellement les scores d'entraînement / training scores ainsi que les estimateurs ajustés) en plus du score de test.

Pour l'évaluation d'une seule metrics, où le paramètre de Scoring/notation est une chaîne de caractères 'string', appelable ou None, les clés seront :
        
        - ['test_score', 'fit_time', 'score_time']

Et pour l'évaluation de metrics multiples, la valeur retournée est un dict avec les clés suivantes :

    ['test_<scorer1_name>', 'test_<scorer2_name>', 'test_<scorer...>', 'fit_time', 'score_time']

return_train_score est défini à False par défaut pour gagner du temps de calcul. Pour évaluer les scores sur l'ensemble d'entraînement également, il doit être défini à True.

In [ ]:
## CRÉER X et y
X = df.drop('sales',axis=1)
y = df['sales']

# TRAIN TEST SPLIT / Fractionnement Entraînement test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# SCALE DATA / Mise à l'échelle des données
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
model = Ridge(alpha=100)

In [ ]:
from sklearn.model_selection import cross_validate

In [ ]:
# Options de SCORING :
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_validate(model,X_train,y_train,
                         scoring=['neg_mean_absolute_error','neg_mean_squared_error','max_error'],cv=5)

In [ ]:
scores

{'fit_time': array([0.00339603, 0.00101757, 0.0009346 , 0.00115156, 0.00112581]),
 'score_time': array([0.00143409, 0.00085998, 0.00165272, 0.00083232, 0.00082564]),
 'test_max_error': array([ -6.44988486,  -5.58926073, -10.33914027,  -6.61950405,
         -7.75578515]),
 'test_neg_mean_absolute_error': array([-2.31243044, -1.74653361, -2.56211701, -2.01873159, -2.27951906]),
 'test_neg_mean_squared_error': array([ -9.32552967,  -4.9449624 , -11.39665242,  -7.0242106 ,
         -8.38562723])}

In [ ]:
pd.DataFrame(scores)

,fit_time,score_time,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_max_error
0,0.003396,0.001434,-2.312430,-9.325530,-6.449885
1,0.001018,0.000860,-1.746534,-4.944962,-5.589261
2,0.000935,0.001653,-2.562117,-11.396652,-10.339140
3,0.001152,0.000832,-2.018732,-7.024211,-6.619504
4,0.001126,0.000826,-2.279519,-8.385627,-7.755785


In [ ]:
pd.DataFrame(scores).mean()

fit_time                        0.001525
score_time                      0.001121
test_neg_mean_absolute_error   -2.183866
test_neg_mean_squared_error    -8.215396
test_max_error                 -7.350715
dtype: float64

**Ajustement du modèle en fonction des metrics**

In [ ]:
model = Ridge(alpha=1)

In [ ]:
# Options de SCORING :
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_validate(model,X_train,y_train,
                         scoring=['neg_mean_absolute_error','neg_mean_squared_error','max_error'],cv=5)

In [ ]:
pd.DataFrame(scores).mean()

fit_time                        0.001629
score_time                      0.001127
test_neg_mean_absolute_error   -1.319685
test_neg_mean_squared_error    -3.344839
test_max_error                 -5.161145
dtype: float64

**Évaluation finale (on ne peut plus modifier les paramètres après cela !)**

In [ ]:
# Il faut d'abord ajuster le modèle !
model.fit(X_train,y_train)

Ridge(alpha=1)

In [ ]:
y_final_test_pred = model.predict(X_test)

In [ ]:
mean_squared_error(y_test,y_final_test_pred)

2.319021579428752